In [1]:
import cv2
import numpy as np


In [2]:

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image


def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image


In [6]:
pip install cython

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI


  Cloning https://github.com/philferriere/cocoapi.git to c:\users\robert~1\appdata\local\temp\pip-install-7o6742r1\pycocotools
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-win_amd64.whl size=79866 sha256=ee4aca1cecbfa51f9d7d2e30781f3627a8b5154986538dc7c2bfaf9ec11631ce
  Stored in directory: C:\Users\ROBERT~1\AppData\Local\Temp\pip-ephem-wheel-cache-_cu7ucfd\wheels\69\2b\12\2fa959e49f73d26cff202c2f4e5079096c9c57c8a8509fd75c
Successfully built pycocotools
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/philferriere/cocoapi.git 'C:\Users\ROBERT~1\AppData\Local\Temp\pip-install-7o6742r1\pycocotools'


In [4]:
pip install opencv-python 

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install opencv-contrib-python 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Accès refusé: 'd:\\anaconda\\envs\\mrcnn\\Lib\\site-packages\\cv2\\cv2.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
conda remove opencv


Solving environment: ...working... failed

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are missing from the target environment:
  - opencv




In [5]:
conda install -c menpo opencv


Solving environment: ...working... done

## Package Plan ##

  environment location: d:\anaconda\envs\mrcnn

  added / updated specs:
    - opencv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    hdf5-1.8.20                |       hac2f561_1        26.7 MB
    libopencv-3.4.2            |       h20b85fd_0        37.4 MB
    libtiff-4.1.0              |       h56a325e_0         997 KB
    mkl-service-2.3.0          |   py36hb782905_0         200 KB
    mkl_fft-1.0.15             |   py36h14836fe_0         137 KB
    mkl_random-1.1.0           |   py36h675688f_0         271 KB
    numpy-1.17.4               |   py36h4320e6b_0           5 KB
    numpy-base-1.17.4          |   py36hc3f5095_0         4.8 MB
    opencv-3.4.2               |   py36h40b0b35_0          10 KB
    py-opencv-3.4.2            |   py36hc319ecb_0         1.5 MB
    six-1.13.0                 |           py36_0        

In [7]:
pip install --upgrade pip


Requirement already up-to-date: pip in d:\anaconda\envs\mrcnn\lib\site-packages (19.3.1)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install opencv-contrib-python

  Using cached https://files.pythonhosted.org/packages/bf/2a/954507b9308378cc0de3082bf0716dcfd915fe6f9c1bb786f0c6c2b219d2/opencv_contrib_python-4.1.2.30-cp36-cp36m-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Accès refusé: 'd:\\anaconda\\envs\\mrcnn\\Lib\\site-packages\\cv2\\cv2.cp36-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [3]:


if __name__ == '__main__':
    """
        test everything
    """
    import os
    import sys
    from samples.coco import coco
    from mrcnn import utils
    from mrcnn import model as modellib
    import matplotlib.pyplot as plt

    ROOT_DIR = os.getcwd()
    MODEL_DIR = os.path.join(ROOT_DIR, "logs")
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
    if not os.path.exists(COCO_MODEL_PATH):
        utils.download_trained_weights(COCO_MODEL_PATH)

    class InferenceConfig(coco.CocoConfig):
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1

    config = InferenceConfig()
    config.display()

    model = modellib.MaskRCNN(
        mode="inference", model_dir=MODEL_DIR, config=config
    )
    model.load_weights(COCO_MODEL_PATH, by_name=True)
    class_names = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
        'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
        'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
        'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
        'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
        'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
        'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
        'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
        'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
        'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
        'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
        'teddy bear', 'hair drier', 'toothbrush'
    ]

    capture = cv2.VideoCapture(0)

    # these 2 lines can be removed if you dont have a 1080p camera.
#    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
#    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 800)

    while True:
        ret, frame = capture.read()
        results = model.detect([frame], verbose=0)
        r = results[0]
        frame = display_instances(
            frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
        )
#        print(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()

d:\anaconda\envs\mrcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda\envs\mrcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda\envs\mrcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda\envs\mrcnn\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passing (type


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
pip uninstall opencv-python